In [3]:
import os
import random
import shutil

# Define paths
main_folder = '/home/shuruyi/Documents/qgis/google_satelite/extracted_images/'
train_folder = '/home/shuruyi/Documents/qgis/google_satelite/train'
test_folder = '/home/shuruyi/Documents/qgis/google_satelite/test'

# Create train and test directories if they don't exist
os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

# List all TIFF files in the main folder
tif_files = [f for f in os.listdir(main_folder) if f.endswith('.tif')]

# Define the ratio for train-test split (e.g., 80-20 split)
train_ratio = 0.8
num_train = int(len(tif_files) * train_ratio)

# Randomly shuffle the list of files
random.shuffle(tif_files)

# Move files to train folder
for file in tif_files[:num_train]:
    src = os.path.join(main_folder, file)
    dst = os.path.join(train_folder, file)
    shutil.move(src, dst)

# Move files to test folder
for file in tif_files[num_train:]:
    src = os.path.join(main_folder, file)
    dst = os.path.join(test_folder, file)
    shutil.move(src, dst)

In [4]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from PIL import Image
import scipy

# CNN model parameters
img_width, img_height = 150, 150
input_shape = (img_width, img_height, 3)
epochs = 10
batch_size = 32

# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_folder, target_size=(img_width, img_height),
                                                    batch_size=batch_size, class_mode='binary')

test_generator = test_datagen.flow_from_directory(test_folder, target_size=(img_width, img_height),
                                                  batch_size=batch_size, class_mode='binary')

# Define CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_generator, steps_per_epoch=train_generator.samples // batch_size,
          epochs=epochs, validation_data=test_generator,
          validation_steps=test_generator.samples // batch_size)

# Save the model
model.save('tree_classifier_model.h5')

Found 1449 images belonging to 2 classes.
Found 336 images belonging to 2 classes.
Epoch 1/10
45/45 [==============================] - 184s 4s/step - loss: 0.5552 - accuracy: 0.7283 - val_loss: 0.3788 - val_accuracy: 0.8938
Epoch 2/10
45/45 [==============================] - 151s 3s/step - loss: 0.3820 - accuracy: 0.8292 - val_loss: 0.2118 - val_accuracy: 0.9250
Epoch 3/10
45/45 [==============================] - 152s 3s/step - loss: 0.3444 - accuracy: 0.8440 - val_loss: 0.2095 - val_accuracy: 0.9344
Epoch 4/10
45/45 [==============================] - 158s 4s/step - loss: 0.3158 - accuracy: 0.8659 - val_loss: 0.1717 - val_accuracy: 0.9344
Epoch 5/10
45/45 [==============================] - 158s 3s/step - loss: 0.2944 - accuracy: 0.8779 - val_loss: 0.2469 - val_accuracy: 0.8875
Epoch 6/10
45/45 [==============================] - 242s 5s/step - loss: 0.2618 - accuracy: 0.8970 - val_loss: 0.3255 - val_accuracy: 0.8344
Epoch 7/10
45/45 [==============================] - 270s 6s/step - loss

/home/shuruyi/Documents/git/IC/.venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
